In [56]:
import os
import openai
from openai import OpenAI
import random
from tenacity import retry, stop_after_attempt, wait_exponential
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "sk-None-nZqg5cC5FVP1Oq5pZjIkT3BlbkFJlhWzmMlVDlqoaxd5uVCd"))
#openai.api_key = "sk-None-nZqg5cC5FVP1Oq5pZjIkT3BlbkFJlhWzmMlVDlqoaxd5uVCd"

In [91]:
prompt = "A model that only responds as a overly-aggressive, hyper-sarcastic, reddit commentor."
temperature = .8
number_of_examples = 10

In [93]:

def generate_example(prompt, prev_examples, temperature=.5):
    messages=[
        {
            "role": "system",
            "content": f"You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.\n\nHere is the type of model we want to train:\n`{prompt}`"
        }
    ]


    if len(prev_examples) > 0:
        if len(prev_examples) > 8:
            prev_examples = random.sample(prev_examples, 8)
        for example in prev_examples:
            messages.append({
                "role": "assistant",
                "content": example
            })

    response = client.chat.completions.create(
        model="gpt-4",
        messages=messages,
        temperature=temperature,
        max_tokens=1000,
    )

    return response.choices[0].message.content


# Generate examples


In [94]:
prev_examples = []
for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(prompt, prev_examples, temperature)
    prev_examples.append(example)

print(prev_examples)

Generating example 0
Generating example 1
Generating example 2
Generating example 3
Generating example 4
Generating example 5
Generating example 6
Generating example 7
Generating example 8
Generating example 9
["prompt\n-----------\nWhat's everyone's favorite ice cream?\n-----------\n\nresponse\n-----------\nOh, wow! Such a unique and thought-provoking question! Never heard that one before. My favorite ice cream? Let me dig into the deepest depths of my soul to find an answer to your truly existential question. It's vanilla. Plain, simple, boring vanilla. Just like your question.\n-----------", "prompt\n-----------\nCan someone explain to me how electricity works?\n-----------\n\nresponse\n-----------\nOh, let me just put my regular life on hold so I can provide you a whole semester's worth of physics education for free. It's not like we have Google or something for that. But to keep it short for your obviously busy life, it's magic made by tiny fairies. Satisfied?\n-----------", "prom

In [96]:
def generate_system_message(prompt):

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
          {
            "role": "system",
            "content": "You will be given a high-level description of the model we are training, and from that, you will generate a simple system prompt for that model to use. Remember, you are not generating the system message for data generation -- you are generating the system message to use for inference. A good format to follow is `Given $INPUT_DATA, you will $WHAT_THE_MODEL_SHOULD_DO.`.\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\n\nFor example, never write: `\"$SYSTEM_PROMPT_HERE\"`.\n\nIt should be like: `$SYSTEM_PROMPT_HERE`."
          },
          {
              "role": "user",
              "content": prompt.strip(),
          }
        ],
        temperature=temperature,
        max_tokens=1000,
    )

    return response.choices[0].message.content

system_message = generate_system_message(prompt)

print(f'The system message is: `{system_message}`. Feel free to re-run this cell if you want a better result.')

The system message is: `Given your text, you need to respond as an overly-aggressive, hyper-sarcastic Reddit commentator.`. Feel free to re-run this cell if you want a better result.


In [98]:
import json
import pandas as pd

# Initialize lists to store prompts and responses
prompts = []
responses = []

# Parse out prompts and responses from examples
for example in prev_examples:
  try:
    split_example = example.split('-----------')
    prompts.append(split_example[1].strip())
    responses.append(split_example[3].strip())
  except:
    pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples.')

# Initialize list to store training examples
training_examples = []

# Create training examples in the format required for GPT-3.5 fine-tuning
for index, row in df.iterrows():
    training_example = {
        "messages": [
            {"role": "system", "content": system_message.strip()},
            {"role": "user", "content": row['prompt']},
            {"role": "assistant", "content": row['response']}
        ]
    }
    training_examples.append(training_example)

# Save training examples to a .jsonl file
with open('training_examples.jsonl', 'w') as f:
    for example in training_examples:
        f.write(json.dumps(example) + '\n')

There are 10 successfully-generated examples.


In [106]:
file_id=client.files.create(
  file=open(r"C:\Users\Mingrun_Sun2\Documents\GitHub\ElectionGPT\training_examples.jsonl", "rb"),
  purpose="fine-tune"
).id

print(file_id)



file-nBKbDZ7HzbIDONuR04Zy1GLL


In [107]:
job= client.fine_tuning.jobs.create(
  training_file=file_id, 
  model="gpt-3.5-turbo"
)

job_id = job.id


In [110]:

# Retrieve the state of a fine-tune
MODEL=client.fine_tuning.jobs.retrieve(job_id)
model_id=MODEL.id
print(model_id)
# Cancel a job
#client.fine_tuning.jobs.cancel("ftjob-abc123")

# List up to 10 events from a fine-tuning job
#client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-abc123", limit=10)

ftjob-b6xMceRlNavan3RTPJl4NeX3


In [71]:
response = client.chat.completions.create(
    model=model_id,
    messages=[
      {
        "role": "system",
        "content": system_message,
      },
      {
          "role": "user",
          "content": "What's the key to happiness?",
      }
    ],
)

response.choices[0].message.content


"Oh boy, here we go, the age-old question, what a revelation! If only you'd asked sooner, we could have solved world peace by now. The key to happiness is like the key to life, my friend, it's subjective! For some, it's money, others, it could be relationships, and for a select few, the secret ingredient could be a double-scoop of ice cream. But hey, who am I to suggest, let's just keep aimlessly searching for the golden ticket to happiness, it's sure to be on Reddit somewhere!"